In [37]:
import re
from functools import reduce
import itertools
from collections import Counter
import numpy as np
from pprint import pprint
from dataclasses import dataclass, field


north, west, east, south = (0,-1),(-1,0),(1,0),(0,1)
pipes = {
    '|': (north ,south),
    "-": (west, east),
    "L": (north, east),
    "J": (north, west),
    "7": (south, west),
    "F": (south, east),
    "S": (south,east,north,west)
}

def navigate(layout,clockwise=False):
    xmax = len(layout[0])-1
    ymax = len(layout)-1
    locations = []

    @dataclass
    class Node():
        x: int
        y:int
        pipe:str


        def connects(self, n):
            if '.' in [self.pipe,n.pipe]:
                # print('skipping .')
                return False
            d = np.array((n.x,n.y))-np.array((self.x,self.y))
            # print(d, pipes[self.pipe], pipes[n.pipe])
            return (d[0],d[1]) in pipes[self.pipe] and (-d[0],-d[1]) in pipes[n.pipe]

        def neighbors(self):
            x,y = self.x, self.y
            candidates = [
                [x-1,y],
                [x,y+1],
                [x+1,y],
                [x,y-1]
            ]
            return (Node(xi,yi,layout[yi][xi]) for xi, yi in filter(lambda k: 0 <=k[0] <=xmax and 0<=k[1]<=ymax,candidates))

        def print_surroundings(self):
            for i in [l[self.x-1:self.x+2] for l in layout[self.y-1:self.y+2]]:
                print(i)

        def get_next(self):
            # Start at entry or top
            #self.print_surroundings()
            checks = itertools.cycle(self.neighbors())
            trash = self
            #print(list(self.neighbors()))
            i = 0
            while not np.array_equal(trash, locations[-1] if len(locations) > 0 else Node(self.x,self.y-1,layout[self.y-1][self.x])):

                trash = next(checks)
                i+=1
                #if i %10 == 0: print(i)
                if i>10:
                    if self.pipe=='S':
                        break
                    else: #self.print_surroundings()
                        raise Exception('Stuck!')

            checks = list(zip(checks,range(4)))
            # print(checks)
            for c,i in checks:
                if i == 5: break
                if self.connects(c): return c
            #self.print_surroundings()
            raise Exception('Stuck!')


    # print(list(Node(1,1,"S").neighbors()))
    assert Node(1,1,'S').connects( Node(1,0,'|'))
    assert not Node(1,1,'S').connects( Node(0,1,'|'))
    assert not Node(1,1,'S').connects( Node(0,1,'.'))
    assert Node(1,1,'S').connects( Node(2,1,'J'))
    assert Node(0,0,'-').connects( Node(1,0,'-'))
    assert Node(17,104,'-').connects( Node(18,104,'-'))
    assert Node(0,0,'-').connects( Node(1,0,'7'))
    assert Node(0,0,'-').connects( Node(-1,0,'F'))
    assert Node(0,0,'F').connects( Node(1,0,'7'))
    assert Node(0,0,'F').connects( Node(1,0,'-'))
    assert Node(0,0,'F').connects( Node(1,0,'J'))
    assert Node(0,0,'|').connects( Node(0,1,'|'))

    #assert Node(1,1,'S').get_next() == Node(1,2,'|')


    def get_starting_point():
        for y, l in enumerate(layout):
            if "S" in l:
                return Node(l.index('S'),y,'S')


    current_node = get_starting_point()

    print('starting at',current_node)
    steps = 0

    while True:
        next_node = current_node.get_next()
        steps+=1
        locations.append(current_node)
        current_node = next_node
        #print(current_node)
        if current_node.pipe =="S" or steps > 10**6:
            locations.append(current_node)
            break
    else:
        return

    print("Loop found!")
    def get_furthest_point(locations):
        distances_cl = [i for i,_ in enumerate(locations)]
        distances_cc = [i for i,_ in enumerate(reversed(locations))]

        distances = [ min(a,b) for a,b, in zip(distances_cl, reversed(distances_cc)) ]
        #print(distances)
        # print(len(locations), len(locations)//2)
        return locations[distances.index(max(distances))], max(distances)

    return locations, get_furthest_point(locations)


def get_input(n):
    with open('input_'+n+'.txt', 'r') as infile:
        data = infile.read().strip()
    return data


def parse_input(puzzle):
    return puzzle.split('\n')


sample1 = """.....
.S-7.
.|.|.
.L-J.
....."""

sample2 = """.....
.S-7.
.L7|.
.FJ|.
.L-J."""

sample3 = """..F7.
.FJ|.
SJ.L7
|F--J
LJ..."""


def solve1(puzzle):
    layout = parse_input(puzzle)
    # for l in layout: print(l)
    return navigate(layout)

print(solve1(sample1)[-1])
print(solve1(sample2)[-1])
print(solve1(sample3)[-1])



starting at navigate.<locals>.Node(x=1, y=1, pipe='S')
Loop found!
(navigate.<locals>.Node(x=3, y=3, pipe='J'), 4)
starting at navigate.<locals>.Node(x=1, y=1, pipe='S')
Loop found!
(navigate.<locals>.Node(x=2, y=4, pipe='-'), 6)
starting at navigate.<locals>.Node(x=0, y=2, pipe='S')
Loop found!
(navigate.<locals>.Node(x=4, y=2, pipe='7'), 8)


In [38]:
puzzle = get_input('10')

In [39]:
print(solve1(puzzle)[-1])

starting at navigate.<locals>.Node(x=104, y=18, pipe='S')
Loop found!
(navigate.<locals>.Node(x=36, y=124, pipe='7'), 6714)


In [40]:
sample5 = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L"""

sample4 = """.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ..."""

sample6 = """F------7
S......|
L---7..|
F---J..|
L------J"""


def n_blobs(line):
    if match := re.findall('1+',line):
        return len(match)
    return 0


assert n_blobs('1000010') == 2
assert n_blobs('00111100010') == 2
assert n_blobs('00111100010') == 2
assert n_blobs('0000010') == 1
assert n_blobs('000') == 0
assert n_blobs('1') == 1
assert n_blobs('') == 0
assert n_blobs('00111000110010') == 3
assert n_blobs('001110001100110') == 3



In [54]:
import numpy as np

# tip from:
# https://www.reddit.com/r/adventofcode/comments/18f1sgh/comment/kcr8tyf/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
def shoelace(x_y):
    x_y = np.array(x_y)
    x_y = x_y.reshape(-1,2)

    x = x_y[:,0]
    y = x_y[:,1]

    S1 = np.sum(x*np.roll(y,-1))
    S2 = np.sum(y*np.roll(x,-1))

    area = .5*np.absolute(S1 - S2)

    return area


def solve2(puzzle):
    locations, _ = solve1(puzzle)
    loop = set((li.x, li.y) for li in locations)

    area = shoelace([(li.x,li.y) for li in locations])
    internal_vertices = area - len(loop)/2+1
    return int(internal_vertices)

sample7="""F---7.
|...|.
S...|.
L---J."""
print(solve2(sample6))
print(solve2(sample7))
print(solve2(sample4))
print(solve2(sample5))


starting at navigate.<locals>.Node(x=0, y=1, pipe='S')
Loop found!
10
starting at navigate.<locals>.Node(x=0, y=2, pipe='S')
Loop found!
6
starting at navigate.<locals>.Node(x=12, y=4, pipe='S')
Loop found!
8
starting at navigate.<locals>.Node(x=4, y=0, pipe='S')
Loop found!
10


In [55]:
validation_sample="""...........
.F-------7.
.SF-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
..........."""
assert solve2(validation_sample) == 4
solve2(puzzle)

starting at navigate.<locals>.Node(x=1, y=2, pipe='S')
Loop found!
starting at navigate.<locals>.Node(x=104, y=18, pipe='S')
Loop found!


429